In [1]:
import numpy as np
import open3d as o3d
import torch

from superprimitive_fusion.scanner import capture_spherical_scans
from superprimitive_fusion.utils import bake_uv_to_vertex_colours
from superprimitive_fusion.mesh_fusion_utils import get_mesh_components, show_mesh_boundaries
from superprimitive_fusion.mesh_fusion import fuse_meshes

In [5]:
names = (
    ('mustard-bottle', 'mustard-bottle.ply'),
    ('table', 'table.obj'),
    ('power-drill', 'power-drill.obj'),
    ('bleach', 'bleach.obj'),
    ('pitcher', 'pitcher.obj'),
    ('mug', 'mug.obj'),
    ('extra-large-clamp', 'extra-large-clamp-leaning.obj'),
)

meshes = []
for foldername,filename in names:
    print(f'Getting the {filename.split('.')[0]}')
    
    mesh = o3d.io.read_triangle_mesh(f"../data/posed-meshes/{foldername}/{filename}", enable_post_processing=True)

    bake_uv_to_vertex_colours(mesh)
    mesh.compute_vertex_normals()

    meshes.append(mesh)

Getting the mustard-bottle
No texture / UVs found; skipping bake
Getting the table
Getting the power-drill
Getting the bleach
Getting the pitcher
Getting the mug
Getting the extra-large-clamp-leaning


In [ ]:
o3d.visualization.draw_geometries(meshes)

: 

In [ ]:
# {
# 	"class_name" : "ViewTrajectory",
# 	"interval" : 29,
# 	"is_loop" : false,
# 	"trajectory" : 
# 	[
# 		{
# 			"boundingbox_max" : [ 0.89999997615814209, 0.44999998807907104, 0.97050297260284424 ],
# 			"boundingbox_min" : [ -0.89999997615814209, -0.44999998807907104, 0.0 ],
# 			"field_of_view" : 60.0,
# 			"front" : [ 0.033181979928996505, 0.89223756678007571, 0.45034551250607779 ],
# 			"lookat" : [ 0.0088310242409319843, 0.0013222086626097952, 0.81265236644826389 ],
# 			"up" : [ -0.0093729695001894316, -0.45029602336870767, 0.89283012873730749 ],
# 			"zoom" : 0.11999999999999962
# 		}
# 	],
# 	"version_major" : 1,
# 	"version_minor" : 0
# }

In [ ]:
import numpy as np
import open3d as o3d

def render_rgb_view(
    mesh: o3d.geometry.TriangleMesh,
    cam_centre=(0.3, 0.3, 0.0),
    look_dir=(0.0, 0.0, 0.0),
    *,
    width_px: int = 360,
    height_px: int = 240,
    fov: float = 70.0,
    bg_rgba=(1.0, 1.0, 1.0, 1.0),  # white background
) -> np.ndarray:
    """Render an RGB image of `mesh` from a given camera pose."""
    renderer = o3d.visualization.rendering.OffscreenRenderer(width_px, height_px)

    # Simple material: unlit if vertex colors exist, otherwise lit shading
    mtl = o3d.visualization.rendering.MaterialRecord()
    mtl.shader = "defaultUnlit" if mesh.has_vertex_colors() else "defaultLit"
    renderer.scene.add_geometry("mesh", mesh, mtl)
    renderer.scene.set_background(bg_rgba)

    # Camera: vertical FOV in degrees, look-at with Z-up
    aspect = width_px / height_px
    cam = renderer.scene.camera
    cam.set_projection(
        fov, aspect, 0.01, 1000.0,
        o3d.visualization.rendering.Camera.FovType.Vertical
    )
    cam.look_at(look_dir, cam_centre, [0, 0, 1])

    # Render to image and return RGB
    img = renderer.render_to_image()
    rgb = np.asarray(img)
    if rgb.ndim == 3 and rgb.shape[2] == 4:  # drop alpha if present
        rgb = rgb[:, :, :3]
    return rgb
